In [1]:
import pandas as pd
import numpy as np

PSEUDONYMISED_PATIENT_ID,AGE_AT_ARRIVAL,GENDER_NATIONAL_DESCRIPTION,SITE,ARRIVAL_DTTM,ARRIVAL_MODE_NATIONAL_CODE,TRIAGE_ASSESSMENT_DTTM,FIRSTDOC_FOR_TREATMENT_DTTM,ADM_REQUEST_DTTM,ADM_REQUEST_LOC_DESCRIPTION,ADMISSION_FLAG,DEPARTURE_DTTM,STREAM_LOCAL_CODE
4484,56,Female,HOSPITAL1,2013-08-04 09:59:00,2.0,2013-08-01 10:52:00,2013-08-01 12:41:00,,,0,2013-08-03 12:22:00,MIN

# ED

In [2]:
np.random.seed(7)

In [3]:
size = 24

In [4]:
def make_datetime_array(size):
    datetimes = []
    j = 0

    for i in np.arange(size):

        new = pd.datetime(2018,1,1,j,5)
        j+=1
        datetimes.append(new)
    return datetimes

#make_datetime_array(size)
#datetimes

In [5]:
arrival_array = make_datetime_array(size)

In [6]:
ED_data = {
'PSEUDONYMISED_PATIENT_ID':np.arange(0,size),
'AGE_AT_ARRIVAL':np.random.randint(18,100,size),
'GENDER_NATIONAL_DESCRIPTION':[1]*size,
'SITE':['HOSPITAL']*size,
'ARRIVAL_DTTM':arrival_array,
'ARRIVAL_MODE_NATIONAL_CODE':[1]*size,
'TRIAGE_ASSESSMENT_DTTM':[np.datetime64()] * size,
'FIRSTDOC_FOR_TREATMENT_DTTM':[np.datetime64()] * size,
'ADM_REQUEST_DTTM':[np.datetime64()] * size,
'ADM_REQUEST_LOC_DESCRIPTION':['inpatient']*size,
'ADMISSION_FLAG':[1]*size,
'DEPARTURE_DTTM':[np.datetime64()] * size,
'STREAM_LOCAL_CODE':['MAJORS']* size,
}

In [7]:
dfED = pd.DataFrame(data=ED_data)
dfED.head()

,ADMISSION_FLAG,ADM_REQUEST_DTTM,ADM_REQUEST_LOC_DESCRIPTION,AGE_AT_ARRIVAL,ARRIVAL_DTTM,ARRIVAL_MODE_NATIONAL_CODE,DEPARTURE_DTTM,FIRSTDOC_FOR_TREATMENT_DTTM,GENDER_NATIONAL_DESCRIPTION,PSEUDONYMISED_PATIENT_ID,SITE,STREAM_LOCAL_CODE,TRIAGE_ASSESSMENT_DTTM
0,1,NaT,inpatient,65,2018-01-01 00:05:00,1,NaT,NaT,1,0,HOSPITAL,MAJORS,NaT
1,1,NaT,inpatient,86,2018-01-01 01:05:00,1,NaT,NaT,1,1,HOSPITAL,MAJORS,NaT
2,1,NaT,inpatient,43,2018-01-01 02:05:00,1,NaT,NaT,1,2,HOSPITAL,MAJORS,NaT
3,1,NaT,inpatient,85,2018-01-01 03:05:00,1,NaT,NaT,1,3,HOSPITAL,MAJORS,NaT
4,1,NaT,inpatient,41,2018-01-01 04:05:00,1,NaT,NaT,1,4,HOSPITAL,MAJORS,NaT


#### fill blank datetime columns

In [8]:
dfED.TRIAGE_ASSESSMENT_DTTM = dfED.ARRIVAL_DTTM + pd.Timedelta(10,'m')
dfED.FIRSTDOC_FOR_TREATMENT_DTTM = dfED.ARRIVAL_DTTM + pd.Timedelta(10,'m')
dfED.ADM_REQUEST_DTTM = dfED.ARRIVAL_DTTM + pd.Timedelta(45,'m')
dfED.DEPARTURE_DTTM = dfED.ARRIVAL_DTTM + pd.Timedelta(60,'m')

In [17]:
dfED.to_csv('./SimpleED.csv',index=False)

# IP

In [9]:
size = 3

In [10]:
arrival_array = make_datetime_array(size)

In [11]:
IP_data = {
'PSEUDONYMISED_PATIENT_ID':np.arange(0,size) + 100,
'SPELL_ID':np.arange(0,size),
'AGE_AT_ARRIVAL':np.random.randint(18,100,size),
'GENDER_NATIONAL_DESCRIPTION':[1] * size,
'SITE':['HOSPITAL']*size,
'ADM_DTTM':arrival_array,
'ADM_SOURCE_CODE':[1] * size,
'ADM_METHOD':[1] * size,
'DIS_DTTM':[np.datetime64()] * size,
'DIS_DESTINATION':['home'] * size,
'DIS_METHOD':[1] * size,
'MOVE_NUMBER':[1] * size,
'MOVE_START':[np.datetime64()] * size,
'MOVE_END':[np.datetime64()] * size,
'MOVE_NAME':['WARD'] * size,
'SPELL_LOS':[np.nan]*size
}

In [12]:
dfIP = pd.DataFrame(data=IP_data)
dfIP.head()

,ADM_DTTM,ADM_METHOD,ADM_SOURCE_CODE,AGE_AT_ARRIVAL,DIS_DESTINATION,DIS_DTTM,DIS_METHOD,GENDER_NATIONAL_DESCRIPTION,MOVE_END,MOVE_NAME,MOVE_NUMBER,MOVE_START,PSEUDONYMISED_PATIENT_ID,SITE,SPELL_ID,SPELL_LOS
0,2018-01-01 00:05:00,1,1,87,home,NaT,1,1,NaT,WARD,1,NaT,100,HOSPITAL,0,NaN
1,2018-01-01 01:05:00,1,1,74,home,NaT,1,1,NaT,WARD,1,NaT,101,HOSPITAL,1,NaN
2,2018-01-01 02:05:00,1,1,42,home,NaT,1,1,NaT,WARD,1,NaT,102,HOSPITAL,2,NaN


#### cycle through each entry and create the ward move data

In [15]:
newdf = dfIP.copy()
for index, row in dfIP.iterrows():
#     print(index,row)
    #### input ward start and end to old record
    
    newdf.loc[index,'MOVE_START'] = newdf.loc[index]['ADM_DTTM']
    move_datetime = newdf.loc[index]['ADM_DTTM'] + pd.Timedelta(24,'h')
    newdf.loc[index,'MOVE_END'] = move_datetime
    
    #### make new record with start and end record

    row['MOVE_START'] = move_datetime # set move start

    moveend_datetime = newdf.loc[index]['MOVE_END'] + pd.Timedelta(48,'h') # add time to previous move end
    row.loc['MOVE_END'] = moveend_datetime
    newdf = newdf.append(row)
    newdf.reset_index(inplace=True,drop=True)
    
    #### set DIS_DTTM in all records from largest 'MOVE_END'
    spell_id = newdf.loc[index]['SPELL_ID']
    newdf.loc[newdf[newdf['SPELL_ID'] == spell_id].index,'DIS_DTTM'] = moveend_datetime
    
    
newdf.head()

,ADM_DTTM,ADM_METHOD,ADM_SOURCE_CODE,AGE_AT_ARRIVAL,DIS_DESTINATION,DIS_DTTM,DIS_METHOD,GENDER_NATIONAL_DESCRIPTION,MOVE_END,MOVE_NAME,MOVE_NUMBER,MOVE_START,PSEUDONYMISED_PATIENT_ID,SITE,SPELL_ID,SPELL_LOS
0,2018-01-01 00:05:00,1,1,87,home,2018-01-04 00:05:00,1,1,2018-01-02 00:05:00,WARD,1,2018-01-01 00:05:00,100,HOSPITAL,0,NaN
1,2018-01-01 01:05:00,1,1,74,home,2018-01-04 01:05:00,1,1,2018-01-02 01:05:00,WARD,1,2018-01-01 01:05:00,101,HOSPITAL,1,NaN
2,2018-01-01 02:05:00,1,1,42,home,2018-01-04 02:05:00,1,1,2018-01-02 02:05:00,WARD,1,2018-01-01 02:05:00,102,HOSPITAL,2,NaN
3,2018-01-01 00:05:00,1,1,87,home,2018-01-04 00:05:00,1,1,2018-01-04 00:05:00,WARD,1,2018-01-02 00:05:00,100,HOSPITAL,0,NaN
4,2018-01-01 01:05:00,1,1,74,home,2018-01-04 01:05:00,1,1,2018-01-04 01:05:00,WARD,1,2018-01-02 01:05:00,101,HOSPITAL,1,NaN


In [16]:
newdf.to_csv('./SimpleIP.csv',index=False)

In [177]:
spell_id

0

In [178]:
newdf['SPELL_ID'] == spell_id

0     True
1    False
2    False
3     True
Name: SPELL_ID, dtype: bool

,index,ADM_DTTM,ADM_METHOD,ADM_SOURCE_CODE,AGE_AT_ARRIVAL,DIS_DESTINATION,DIS_DTTM,DIS_METHOD,GENDER_NATIONAL_DESCRIPTION,MOVE_END,MOVE_NAME,MOVE_NUMBER,MOVE_START,PSEUDONYMISED_PATIENT_ID,SITE,SPELL_ID,SPELL_LOS
0,0,2018-01-01 00:05:00,1,1,29,home,NaT,1,1,2018-01-02 00:05:00,WARD,1,2018-01-01 00:05:00,100,HOSPITAL,0,NaN
1,1,2018-01-01 01:05:00,1,1,50,home,NaT,1,1,2018-01-02 01:05:00,WARD,1,2018-01-01 01:05:00,101,HOSPITAL,1,NaN
2,2,2018-01-01 02:05:00,1,1,58,home,NaT,1,1,2018-01-02 02:05:00,WARD,1,2018-01-01 02:05:00,102,HOSPITAL,2,NaN
3,0,2018-01-01 00:05:00,1,1,29,home,NaT,1,1,NaT,WARD,1,2018-01-02 00:05:00,100,HOSPITAL,0,NaN
4,1,2018-01-01 01:05:00,1,1,50,home,NaT,1,1,NaT,WARD,1,2018-01-02 01:05:00,101,HOSPITAL,1,NaN
5,2,2018-01-01 02:05:00,1,1,58,home,NaT,1,1,NaT,WARD,1,2018-01-02 02:05:00,102,HOSPITAL,2,NaN


In [120]:
nrec = dfIP.loc[0]

In [150]:
row['ADM_DTTM'] = 5

In [151]:
row

ADM_DTTM                              5
ADM_METHOD                            1
ADM_SOURCE_CODE                       1
AGE_AT_ARRIVAL                       58
DIS_DESTINATION                    home
DIS_DTTM                            NaT
DIS_METHOD                            1
GENDER_NATIONAL_DESCRIPTION           1
MOVE_END                            NaT
MOVE_NAME                          WARD
MOVE_NUMBER                           1
MOVE_START                          NaT
PSEUDONYMISED_PATIENT_ID            102
SITE                           HOSPITAL
SPELL_ID                              2
SPELL_LOS                           NaN
Name: 2, dtype: object

In [123]:
dfIP.append(nrec)

,ADM_DTTM,ADM_METHOD,ADM_SOURCE_CODE,AGE_AT_ARRIVAL,DIS_DESTINATION,DIS_DTTM,DIS_METHOD,GENDER_NATIONAL_DESCRIPTION,MOVE_END,MOVE_NAME,MOVE_NUMBER,MOVE_START,PSEUDONYMISED_PATIENT_ID,SITE,SPELL_LOS
0,2018-01-01 00:05:00,1,1,74,home,NaT,1,1,NaT,WARD,1,NaT,0,HOSPITAL,NaN
1,2018-01-01 01:05:00,1,1,66,home,NaT,1,1,NaT,WARD,1,NaT,1,HOSPITAL,NaN
2,2018-01-01 02:05:00,1,1,61,home,NaT,1,1,NaT,WARD,1,NaT,2,HOSPITAL,NaN
0,2018-01-01 00:05:00,1,1,74,home,NaT,1,1,NaT,WARD,1,NaT,0,HOSPITAL,NaN
